# Gallery Linker Notebook

## References

- [stashdb-performer-gallery](../../gh-stashapp/CommunityScripts/plugins/stashdb-performer-gallery/stashdb-performer-gallery.py)

In [7]:
import stashapi.log as log
from stashapi.stashapp import StashInterface
from stashapi.stashbox import StashBoxInterface
import os
import sys
import requests
import json
import requests
from pathlib import Path
import base64
from urllib.parse import urlparse
from dataclasses import dataclass, field

In [8]:
per_page = 100
request_s = requests.Session()
stash_boxes = {}
scrapers = {}

In [10]:
# Linking statistics dataclasses
@dataclass
class LinkingStat:
    count: int = 0
    without_galleries: int | None = field(default=None, metadata={"help": "Number of items without galleries"})
    without_performers: int | None = field(default=None, metadata={"help": "Number of items without performers"})
    without_scenes: int | None = field(default=None, metadata={"help": "Number of items without scenes"})


class SceneStats(LinkingStat):
    def __post_init__(self):
        self.without_scenes = None
        if self.without_performers is None:
            self.without_performers = 0
        if self.without_galleries is None:
            self.without_galleries = 0


class PerformerStats(LinkingStat):
    def __post_init__(self):
        self.without_performers = None
        if self.without_scenes is None:
            self.without_scenes = 0
        if self.without_galleries is None:
            self.without_galleries = 0


class GalleryStats(LinkingStat):
    def __post_init__(self):
        self.without_galleries = None
        if self.without_performers is None:
            self.without_performers = 0
        if self.without_scenes is None:
            self.without_scenes = 0


@dataclass
class LinkingTotals:
    scenes: SceneStats
    performers: PerformerStats
    galleries: GalleryStats



In [16]:
def get_totals(stash:StashInterface) -> LinkingTotals:
    totals = LinkingTotals(scenes=SceneStats(), performers=PerformerStats(), galleries=GalleryStats())
    totals.galleries.count = stash.find_galleries(get_count=True)[0]
    # galleries_without_scenes = stash.find_galleries(
    #     filter={"scene_id": {"is_null": True}}, get_count=True
    # )
    return totals

stash = StashInterface()
config = stash.get_configuration()
print(json.dumps(config, indent=4,separators=(',', ': ')))
totals = get_totals(stash)
print(totals)

dUsing stash (v0.28.1-0) endpoint at http://localhost:9999/graphql


{
    "general": {
        "stashes": [
            {
                "path": "/data/pictures",
                "excludeVideo": true,
                "excludeImage": false
            },
            {
                "path": "/data/scenes",
                "excludeVideo": false,
                "excludeImage": true
            }
        ],
        "databasePath": "/root/.stash/stash-go.sqlite",
        "backupDirectoryPath": "",
        "generatedPath": "/generated/",
        "metadataPath": "/metadata/",
        "configFilePath": "/root/.stash/config.yml",
        "scrapersPath": "/root/.stash/scrapers",
        "pluginsPath": "/root/.stash/plugins",
        "cachePath": "/cache/",
        "blobsPath": "/root/.stash/blobs",
        "blobsStorage": "FILESYSTEM",
        "ffmpegPath": "",
        "ffprobePath": "",
        "calculateMD5": false,
        "videoFileNamingAlgorithm": "OSHASH",
        "parallelTasks": 1,
        "previewAudio": true,
        "previewSegments": 12,
        

In [5]:
def build_default_config() -> dict:
    """Build default Stash connection configuration."""
    return {"scheme": "http", "host": "localhost", "port": "9999"}


def update_config_from_params(config: dict, stash_url: str | None, api_key: str | None) -> None:
    """Update configuration with provided URL and API key."""
    if stash_url:
        parsed = urlparse(stash_url)
        config.update(
            {
                "scheme": parsed.scheme or "http",
                "host": parsed.hostname or "localhost",
                "port": str(parsed.port or 9999),
            }
        )
    if api_key:
        config["ApiKey"] = api_key

In [ ]:
def get_num_scenes_without_galleries(stash: StashInterface) -> int:
    """Retrieve the number of scenes without associated galleries."""
    scenes = []
    page = 1
    while True:
        result = stash.find_scenes(
            filter={"gallery_id": {"is_null": True}}, get_count=True
        )
        # scenes.extend(result["scenes"])
        if len(result["scenes"]) < per_page:
            break
        page += 1
    return scenes

In [ ]:
# json_input = json.loads(sys.stdin.read())
stash = StashInterface(conn=)
config = stash.get_configuration()["plugins"]
settings = {
    
}